# Notebook 2: Quantifying an image from Listmode data.

In this notebook we will use Python functions to:
- Use the iQID histogram and offsets data to identify issues with acquisition.
- Use the iQID histogram to quantify the activity in an image.
- Apply a decay correction to the time of sample acquisition.

## Import packages

First, move your working directory upwards one so you can access source code in iqid/. This step is necessary because I haven't set up iqid as an installable package (i.e. via pip or conda), reflecting the fact that it's pretty informal and (permanently) in development.

In [1]:
cd ..

C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas


In [2]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

%load_ext autoreload
%autoreload 2
from iqid import helper as iq # helper functions such as plotting parameters and other things
from iqid import process_object as po # class to handle listmode data is contained here.

pltmap = iq.set_plot_parms() # set default plot settings (feel free to change this)

C:\Users\Robin\anaconda3\envs\beq\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Load data
- Follows the procedure outlined in Demo Notebook 1.

In [3]:
%%time

# enter the path to your iQID data on your machine
datdir = r"C:\Users\Robin\Documents\Cal\iQID\git_branches\iqid-alphas\data_sample\File_Structure_Sample"
fname = iq.natural_sort(glob.glob(os.path.join(datdir, "Listmode", "*Compressed_Processed_Listmode.dat")))[-1]
cdat = po.ClusterData(fname, ftype="processed_lm", c_area_thresh=1)
cim = cdat.image_from_big_listmode(event_fx=1)

CPU times: total: 2.58 s
Wall time: 2.58 s
